<a href="https://colab.research.google.com/github/hwarang97/Pytorch_introduction/blob/main/pytorch_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 신경망 구축

- torch.nn  --> 신경망 구성 요소 제공
- nn.Module --> 모든 신경망 계층 다룸

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader # 데이터셋을 이터러블 객체로 만드는데 사용
from torchvision import datasets, transforms # datasets : 데이터셋 제공

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device') # gpu 사용 가능

Using cuda device


## 클래스 정의하기

1. 신경망 모델을 nn.Module의 하위 클래스로 선언
2. __init__ 에서 신경망 계층들을 초기화
3. nn.Module 을 상속받은 모든 클래스는 forward 메소드로 입력 데이터로 연산 수행

In [5]:
class NN(nn.Module):
  def __init__(self):
    super(NN, self).__init__() # super 매개변수로 (하위클래스 이름, 하위클래스 객체) 가 넘겨진다 
                               # 보통 부모 - 자식관계이므로, 생략하면 자식이 부모를 호출한다고 인식하고 생략되기도 함
                               # 만약 조부모(A) - 부모(B) - 자식(C) 관계라고 한다면
                               # 자식 선언시 매개변수를 생략하면 부모클래스를 호출하는것이 됨
                               # 그러나 명시적으로 자식클래스에서 조부모(A)를 매개변수로 넣으면 super(A, self)
                               # 부모(B) 클래스를 건너뛰고 조부모(A) 클래스를 호출하는 것이 됨
                               # 즉, 상황에 따른 설정이 가능하다는 것!!

    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512), # 28*28 개의 픽셀값을 입력받음 --> 512개의 뉴런에게 전달
        nn.ReLU(),            
        nn.Linear(512,512),    # 512개 뉴런 --> 512개 뉴런
        nn.ReLU(),
        nn.Linear(512,10),     # 512개 뉴런 --> 10개 뉴런 (분류할 이미지가 10종류라서)
    )

  def forward(self, x):                # 실제 모델 연산 구현
    x = self.flatten(x)                # flatten 실행
    logits = self.linear_relu_stack(x) # 설정한 linear, relu 연산 실행
    return logits                      # 각 클래스별로 logit 값이 나옴 --> softmax 함수에 넣어서 확률로 변환

In [6]:
model = NN().to(device)
print(model)

NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [26]:
# 임의의 이미지를 만들어서 작동시켜보기
X = torch.rand(1, 28, 28, device=device)
logits = model(X)                       # 각 클래스별 로짓값, tensor([[0.0015, 0.0578, ..., -0.0096]])
pred_probab = nn.Softmax(dim=1)(logits) # 각 클래스별 확률 반환, tensor([[0.0992, 0.1050, ... , 0.0981]]) -> 2차원 배열 형식
y_pred = pred_probab.argmax(1)          # 가장 큰 원소를 반환. numpy 계열 함수 <--> argmin 
                                        # dim=1은 열끼리 비교하도록 설정
print(f'Predicted class : {y_pred}')

Predicted class : tensor([6], device='cuda:0')


In [27]:
dir(nn.Softmax)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__constants__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_call_impl',
 '_get_backward_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_maybe_warn_non_full_backward_hook',
 '_named_members',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_version',
 'add_module',
 'apply',
 'bfloat16',
 'buffers',
 'children',
 'cpu',
 'cuda',
 'double',
 'dump_patches',
 'eval',
 'extra_repr',
 'float',
 'forward',
 'get_buffer',
 'get_extra_state',
 'get_parameter',
 'get_submodule'

### Softmax(dim=1)(logits) 조사
- 형식이 생소했고, 왜 이렇게 만들어졌는지 모르겠었음
  - 내가 알던 양식은 객체 선언 시, 매개변수에 다 같이 넣는것 (softmax(logits, dim=1)    

---
- 조사한 것
- SoftMax.__init__(self, dim) : 객체 선언시 받을 수 있는 인자는 최대 2개뿐. (logits는 생성시 같이 사용할 수 없음)
- SoftMax.forward(self, input : Tensor) -> Tensor : 인자를 받아서 softmax 연산을 해주는 것을 보임. 아마도 logits을 받는 함수인듯
- 결론 : nn.Softmax(dim=1) 은 객체를 생성한 것이고, 이후 객체 함수 forward를 이용해서 logits 을 처리한것

## 모델 계층(Layer)
- 신경망에서 어떤 일이 벌어지는지 확인해보기

In [21]:
# 임의의 입력 이미지 생성
input_image = torch.rand(3, 28, 28) # 28*28 이미지 3장
print(input_image.size())

torch.Size([3, 28, 28])


### Flatten
- Flatten 계층
- 2D 이미지 --> 배열

In [22]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### Linear
- 선형 계층
- 뉴런별로 입력값에 가중치와 편향 값을 사용하여 선형방정식 만들어주는 모듈(코드)

In [23]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size()) # tensor([3,20]), 선형 결과값이 이미지당 1개씩 총 3개

torch.Size([3, 20])


### ReLU
- 활성와 함수, 비선형성
- 더 깊이있게 학습시키 위해서는 비선형성이 필요해서 도입됨

In [24]:
print(f'Before ReLU : {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)

tensor([[-0.0698, -0.1330,  0.1825,  0.7029,  0.4213, -0.2612, -0.5774,  0.0167,
          0.0318, -0.2689,  0.3557, -0.3781,  0.2111,  0.2366, -0.1016,  0.3706,
          0.0350, -0.3127,  0.2891,  0.1211],
        [-0.2330, -0.1554,  0.0498,  0.2508,  0.3912,  0.1016, -0.2031, -0.0235,
         -0.1681, -0.4620,  0.5852, -0.3686,  0.0066,  0.0294, -0.1325,  0.5242,
         -0.1223, -0.3144,  0.2957,  0.2386],
        [-0.1887, -0.1668, -0.0645,  0.2120,  0.6192, -0.3331,  0.0113,  0.3721,
         -0.0940, -0.7104,  0.6013, -0.1680,  0.2121,  0.0492, -0.3345,  0.4411,
         -0.0422, -0.1261,  0.3761,  0.2442]], grad_fn=<AddmmBackward0>)